In [1]:
import os

In [2]:
%pwd

'c:\\Users\\49179\\Desktop\\Food_image_classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\49179\\Desktop\\Food_image_classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen =True)
class preparetensorboardconfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    experiment_name: str
    model_name:str
    epochs: int

In [14]:
from Food_Classification.utils.common import read_yaml, create_directory
from Food_Classification.constants import *

class ConfigurationManager:
    def __init__(self,
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directory([self.config.artifacts_root])

    def get_tensorboard_config(self) -> preparetensorboardconfig:

            config = self.config.prepare_tensorboard
            create_directory([config.tensorboard_root_log_dir])

            prepare_tensorboard_config = preparetensorboardconfig(root_dir = config.root_dir,
                                                            tensorboard_root_log_dir = config.tensorboard_root_log_dir,
                                                            experiment_name = self.params.EXPERIMENT_NAME,
                                                            model_name = self.params.MODEL_NAME,
                                                            epochs = self.params.EPOCHS
                                                            )
            
            return prepare_tensorboard_config

In [17]:
%%writefile src/Food_Classification/components/tensorboard.py
from torch.utils.tensorboard import SummaryWriter
import os
from datetime import datetime
from Food_Classification.config.configuration import get_tensorboard_config

class preparetensorboard:
    def __init__(self, config: preparetensorboardconfig):
        self.config = config

    def create_summary_writer(self,experiment_name, model_name, extra: str = None) -> SummaryWriter:
        now = datetime.now()
        timestamp = now.strftime("%Y%m%d-%H%M")

        if extra:
            log_dir = os.path.join(self.config.tensorboard_root_log_dir, timestamp, experiment_name, model_name,extra)
        else:
            log_dir = os.path.join(self.config.tensorboard_root_log_dir, timestamp, experiment_name, model_name)

        return SummaryWriter(log_dir= log_dir)
    
    def get_summary_writer(self):
        self.summary = self.create_summary_writer(experiment_name=self.config.experiment_name,
                                                  model_name= self.config.model_name,
                                                  extra= f"{self.config.epochs} epochs")
        
        return self.summary

Writing src/Food_Classification/components/tensorboard.py


In [16]:
try:
    config = ConfigurationManager()
    tensorboard_config = config.get_tensorboard_config()
    tensorboard = preparetensorboard(tensorboard_config)
    summary_writer = tensorboard.get_summary_writer()
except Exception as e:
    raise e

[2024-05-03 08:08:48,211: INFO: common: yaml file config\config.yaml loaded successfully]
[2024-05-03 08:08:48,213: INFO: common: yaml file params.yaml loaded successfully]
[2024-05-03 08:08:48,214: INFO: common: directory artifacts created successfully]
[2024-05-03 08:08:48,215: INFO: common: directory artifacts/prepare_tensorboard/tensorboard_log_dir created successfully]
